In [ ]:
# Импорт библиотек 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from catboost import CatBoostRegressor


In [ ]:
# Загрузка данных
train_df = pd.read_csv('/kaggle/input/playground-series-s5e10/train.csv')

print(f"Размер датасета: {train_df.shape}")
print(f"\nПервые строки:")
train_df.head()


In [ ]:
# Информация о данных
print("Информация о данных:")
print(train_df.info())
print("\nСтатистика:")
print(train_df.describe())
print("\nПропущенные значения:")
print(train_df.isnull().sum())


In [ ]:
# Анализ целевой переменной
fig, axes = plt.subplots(1, 1, figsize=(14, 5))

axes.hist(train_df['accident_risk'], bins=50, edgecolor='black')
axes.set_title('Распределение вероятности аварии')
axes.set_xlabel('Вероятность аварии')
axes.set_ylabel('Частота')

plt.tight_layout()
plt.show()

print(f"Минимум: {train_df['accident_risk'].min()}")
print(f"Максимум: {train_df['accident_risk'].max()}")
print(f"Среднее: {train_df['accident_risk'].mean():.4f}")


In [ ]:
# Разделение на признаки и целевую переменную
X = train_df.drop(['id', 'accident_risk'], axis=1)
y = train_df['accident_risk']

print("Типы признаков:")
print(X.dtypes)
print("\nКатегориальные признаки:")
categorical_features = X.select_dtypes(include=['object', 'bool']).columns.tolist()
print(categorical_features)
print("\nЧисловые признаки:")
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
print(numerical_features)


In [ ]:
# Разделение на train и validation
# CatBoost может работать с категориальными признаками напрямую
X_train, X_val, y_train, y_val = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42
)

print(f"Train размер: {X_train.shape}")
print(f"Validation размер: {X_val.shape}")

# Преобразуем bool в str для CatBoost
for col in categorical_features:
    X_train[col] = X_train[col].astype(str)
    X_val[col] = X_val[col].astype(str)


In [ ]:
# Создание и обучение модели CatBoost
model = CatBoostRegressor(
    iterations=2000,
    depth=7,
    learning_rate=0.1,
    l2_leaf_reg=3,
    subsample=0.9,
    random_seed=42,
    verbose=50,
    early_stopping_rounds=100,
    cat_features=categorical_features,
    grow_policy='Depthwise',
    bagging_temperature=0.5
)

print("Начало обучения модели...")

model.fit(
    X_train, y_train,
    eval_set=(X_val, y_val)
)

print("\nОбучение завершено!")

In [ ]:
# Предсказания и оценка модели
y_train_pred = model.predict(X_train)
y_val_pred = model.predict(X_val)

print("МЕТРИКИ МОДЕЛИ")

print("\nНА TRAIN:")
print(f"RMSE: {np.sqrt(mean_squared_error(y_train, y_train_pred)):.6f}")
print(f"MAE: {mean_absolute_error(y_train, y_train_pred):.6f}")
print(f"R2 Score: {r2_score(y_train, y_train_pred):.6f}")

print("\nНА VALIDATION:")
print(f"RMSE: {np.sqrt(mean_squared_error(y_val, y_val_pred)):.6f}")
print(f"MAE: {mean_absolute_error(y_val, y_val_pred):.6f}")
print(f"R2 Score: {r2_score(y_val, y_val_pred):.6f}")


In [ ]:
# Важность признаков
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(data=feature_importance, x='importance', y='feature')
plt.title('Важность признаков')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()

print("\nТоп-10 наиболее важных признаков:")
print(feature_importance.head(10))


In [ ]:
# Визуализация предсказаний
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Train
axes[0].scatter(y_train, y_train_pred, alpha=0.3, s=5, color='blue')
axes[0].plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'r--', lw=2)
axes[0].set_xlabel('Фактические значения')
axes[0].set_ylabel('Предсказанные значения')
axes[0].set_title(f'Train: R2={r2_score(y_train, y_train_pred):.4f}')
axes[0].grid(True, alpha=0.3)

# Validation
axes[1].scatter(y_val, y_val_pred, alpha=0.3, s=5, color='green')
axes[1].plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', lw=2)
axes[1].set_xlabel('Фактические значения')
axes[1].set_ylabel('Предсказанные значения')
axes[1].set_title(f'Validation: R2={r2_score(y_val, y_val_pred):.4f}')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Создание submission
try:
    test_df = pd.read_csv('/kaggle/input/playground-series-s5e10/test.csv')
    print(f"Тестовый набор загружен: {test_df.shape}")
    
    # Сохраняем ID
    test_ids = test_df['id']
    
    # Подготовка тестовых данных
    X_test = test_df.drop(['id'], axis=1)
    
    # Преобразуем bool в str для CatBoost
    for col in categorical_features:
        if col in X_test.columns:
            X_test[col] = X_test[col].astype(str)
    
    # Предсказания
    test_predictions = model.predict(X_test)
    
    # Создание submission файла
    submission = pd.DataFrame({
        'id': test_ids,
        'accident_risk': test_predictions
    })
    
    submission.to_csv('submission.csv', index=False)
    print("Предсказания сохранены в submission.csv")
    
    print(f"\nПервые строки submission:")
    print(submission.head(10))
    print(f"\nСтатистика предсказаний:")
    print(submission['accident_risk'].describe())
    
except FileNotFoundError:
    print("Файл test.csv не найден. Пропускаем создание submission.")
